In [ ]:
import torch
import torch
import torch.utils.data
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import TQDMProgressBar

from livecellx.model_zoo.segmentation.sc_correction import CorrectSegNet
import numpy as np
import warnings
import tqdm
torch.manual_seed(237)


In [ ]:
# torch.cuda.is_available(), torch.cuda.current_device(), torch.cuda.device_count()

import torch
import torch
import torch.utils.data
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import TQDMProgressBar

from livecellx.model_zoo.segmentation.sc_correction import CorrectSegNet
from livecellx.model_zoo.segmentation.sc_correction_dataset import CorrectSegNetDataset
torch.manual_seed(237)

In [ ]:
from pathlib import Path
import pandas as pd
# train_dir = Path("./notebook_results/a549_ccp_vim/train_data_v4/")
# test_dir = Path("./notebook_results/a549_ccp_vim/test_data_v4/")
# train_dir = Path("./notebook_results/a549_ccp_vim/train_data_v5/")
# test_dir = Path("./notebook_results/a549_ccp_vim/test_data_v5/")

# train_dir = Path("./notebook_results/a549_ccp_vim/train_data_v8/")
# test_dir = Path("./notebook_results/a549_ccp_vim/test_data_v8/")
# train_csv = train_dir / "train_data.csv"

# train_csv = "./notebook_results/a549_ccp_vim/train_real_td_data/real_underseg_td1_XY8/data.csv"
# train_csv = "./notebook_results/a549_ccp_vim/train_real_td_data/real_overseg_td1_XY9/data.csv"
# train_csv = "./notebook_results/a549_ccp_vim/train_data_v11/test_overseg_dropout/data.csv"
# train_csv = "/home/ken67/LiveCellTracker-dev/notebooks/notebook_results/a549_ccp_vim/test_data_v11/real_overseg_td1_XY5_dropout/data.csv"
train_csv = "/home/ken67/LiveCellTracker-dev/notebooks/notebook_results/a549_ccp_vim/train_data_v11/train_data.csv"
# train_csv = "/home/ken67/LiveCellTracker-dev/notebooks/notebook_results/a549_ccp_vim/test_data_v11/train_data.csv"
# train_csv = "D:\\LiveCellTracker-dev\\datasets\\yaxuan_csn_annotation\\train_data.csv"
# train_csv = "/home/ken67/LiveCellTracker-dev/notebooks/notebook_results/a549_ccp_vim/train_data_v6/synthetic_overseg/data.csv"
# train_csv = "/home/ken67/LiveCellTracker-dev/notebooks/notebook_results/a549_ccp_vim/train_data_v9/synthetic_overseg/data.csv"

# train_csv = r"/home/ken67/LiveCellTracker-dev/notebooks/notebook_results/a549_ccp_vim/train_data_v6/synthetic_overseg/data.csv"
train_df = pd.read_csv(train_csv)

if "subdir" not in train_df.columns:
    train_df["subdir"] = "cur"
# test_df = pd.read_csv(test_dir / "train_data.csv")
train_df[:2]

Check all gt labels: any empty?

In [ ]:
from livecellx.core.datasets import read_img_default

def check_unique_labels(img_paths):
    for path in img_paths:
        # read tiff
        gt_tif = read_img_default(path)
        # print("gt shape:", gt_tif.shape)
        # print("unique labels:", np.unique(gt_tif))
        assert np.unique(gt_tif.flatten()).shape[0] > 1, "#unique labels should be > 1"

# # check all gt label's unique labels
# gt_paths = train_df["gt"].values
# check_unique_labels(gt_paths)

In [ ]:
from livecellx.model_zoo.segmentation.eval_csn import compute_metrics, assemble_train_test_dataset

# model = CorrectSegNet.load_from_checkpoint(r"/home/ken67/LiveCellTracker-dev/notebooks/lightning_logs/version_v11_02/checkpoints/epoch=3819-global_step=0.ckpt")
# train_dataset_eval, val_dataset_eval, test_dataset_eval, whole_dataset_eval = assemble_train_test_dataset(train_df, train_df, model)

In [ ]:
from livecellx.model_zoo.segmentation.eval_csn import assemble_dataset, assemble_train_test_dataset

# split_seed = 237
# dataset = assemble_dataset(train_df, apply_gt_seg_edt = model.apply_gt_seg_edt, exclude_raw_input_bg=model.exclude_raw_input_bg, input_type=model.input_type)
# train_sample_num = int(len(dataset) * 0.8)
# val_sample_num = len(dataset) - train_sample_num
# split_generator = torch.Generator().manual_seed(split_seed)
# train_dataset, val_dataset = torch.utils.data.random_split(
#     dataset, [train_sample_num, val_sample_num], generator=split_generator
# )

# test_dataset = assemble_dataset(test_df, apply_gt_seg_edt = model.apply_gt_seg_edt, exclude_raw_input_bg=model.exclude_raw_input_bg, input_type=model.input_type)
train_dataset = assemble_dataset(train_df, apply_gt_seg_edt = True, exclude_raw_input_bg=False, input_type="raw_aug_duplicate")

In [ ]:
from torchvision import transforms
train_dataset.transform =     transforms.Compose(
        [
            transforms.Resize((412, 412))
        ]
    )

Viz training dataset

In [ ]:
train_dataset.normalize_uint8=False
train_dataset.apply_gt_seg_edt = True

In [ ]:
len(train_dataset), len(train_dataset.raw_img_paths)

In [ ]:
train_dataset[0]["input"].shape

In [ ]:
import matplotlib.pyplot as plt
import random

def viz_sample_only(sample, cmap = "viridis"):
    num_imgs = 7
    size = 10
    fig, axes = plt.subplots(1, num_imgs, figsize=(size * num_imgs, size))
    fig.set_facecolor("white")
    # Remove the spaces between subfigures
    fig.subplots_adjust(wspace=0, hspace=0)

    norm = plt.Normalize(vmin=0, vmax=1)
    ax_idx = 0
    ax = axes[ax_idx]
    ax.imshow(sample["input"][0], cmap = cmap)
    ax.set_title("input: dim0")

    ax_idx += 1
    ax = axes[ax_idx]
    ax.imshow(sample["input"][1], cmap = cmap)
    ax.set_title("input: dim1")

    ax_idx += 1
    ax = axes[ax_idx]
    ax.imshow(sample["input"][2], cmap = cmap)
    ax.set_title("input:dim2")

    # gt
    ax_idx += 1
    ax = axes[ax_idx]
    ax.imshow(sample["gt_mask"][0], cmap = cmap)
    ax.set_title("gt0 seg")

    ax_idx += 1
    ax = axes[ax_idx]
    ax.imshow(sample["gt_mask"][1], cmap = cmap)
    ax.set_title("gt1 seg")

    ax_idx += 1
    ax = axes[ax_idx]
    ax.imshow(sample["gt_mask"][2], cmap = cmap)
    ax.set_title("gt2 seg")

    ax_idx += 1
    ax = axes[ax_idx]
    ax.imshow(sample["gt_label_mask"], cmap = cmap)
    ax.set_title("gt label mask")

    for ax in axes:
        ax.axis("off")

out_dir = Path("./tmp/csn_check_data_v6/")
out_dir.mkdir(exist_ok=True, parents=True)
for i in range(4):
    sample_id = random.randint(0, len(train_dataset) - 1)
    sample = train_dataset[sample_id]
    df_row = train_dataset.raw_df.iloc[sample_id]
    print("source subdir:", df_row["subdir"])
    print("sample id:", sample_id)
    # plt.hist(sample["input"][0].flatten(), bins=100)
    # plt.hist(sample["gt_mask"][1].flatten(), bins=100)
    viz_sample_only(sample, cmap="inferno")
    plt.suptitle(f"sample id: {sample_id}, raw_df subdir: {df_row['subdir']}, raw_path: {df_row['raw']}", fontsize=5)
    # plt.subplots_adjust(top=0.55)
    plt.subplots_adjust(wspace=0, hspace=0)
    plt.axis("off")
    plt.show()
    plt.savefig(out_dir / f"sample_{sample_id}.png")

In [ ]:
for sample in tqdm.tqdm(train_dataset):
    gt_label_mask = sample["gt_label_mask"]
    assert np.unique(gt_label_mask).shape[0] > 0

In [ ]:
from livecellx.model_zoo.segmentation.eval_csn import compute_metrics
import importlib
import livecellx.model_zoo.segmentation.eval_csn
importlib.reload(livecellx.model_zoo.segmentation.eval_csn)

model.cuda()
model.eval()
metrics = livecellx.model_zoo.segmentation.eval_csn.compute_metrics(train_dataset, model, whole_dataset=train_dataset)